In [14]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("test")

import logging
import os

import numpy as np
import pandas as pd

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
logging.getLogger().setLevel(logging.ERROR)
import tensorflow as tf
tf.get_logger().setLevel(logging.ERROR)

import sys
sys.path.append("../../../src")
import json
import os
from functools import partial
from enum import Enum
import numpy as np
from itertools import combinations
import tensorflow_addons  # needed for model import, do not remove
import pandas
from keras.callbacks import LambdaCallback
from matplotlib import pyplot as plt
from numpy import array_equal
from sklearn.metrics import accuracy_score, confusion_matrix
from tensorflow import keras
from toolz import thread_first, thread_last, identity, count, groupby

from toolz import keyfilter
from functools import partial
from sklearn.metrics import accuracy_score, confusion_matrix
from Evaluation import run_ensembles, load_models, flatten_models
from Helpers import create_confusion_matrix_plot_from_csv
from LoadData import CurrentDatasets, get_all_datasets_test_train_np_arrays

DEBUG:root:test


In [31]:

class EnsembleMethods(str, Enum):
    AVERAGE = "average"
    LOGISTIC_AVERAGE = "logistic_average"
    MAJORITY_VOTE = "majority_vote"


class Ensemble(tf.keras.Model):
    """
    Ensemble of multiple keras models. Implements multiple methods for ensembling:
    - averaging
    - logistic averaging
    - majority vote
    """

    def __init__(self, models=None, weights=None, ensemble_type: EnsembleMethods = EnsembleMethods.AVERAGE):
        super(Ensemble, self).__init__()
        if models is None:
            models = []
        if weights is None:
            weights = np.ones(len(models))
        # normalize weights
        weights = np.multiply(weights, 1.0 / sum(weights))
        self.models = models
        self.model_weights = weights
        """
        self.__ensemble_method__ = None
        match ensemble_type:
            case EnsembleMethods.AVERAGE:
                self.__ensemble_method__ = self.__average__
            case EnsembleMethods.LOGISTIC_AVERAGE:
                self.__ensemble_method__ = self.__logistic_average__
            case EnsembleMethods.MAJORITY_VOTE:
                self.__ensemble_method__ = self.__majority_vote__
            case _:
                raise ValueError("Invalid ensemble type")
        """
        ensemble_types = {
            EnsembleMethods.AVERAGE : self.__average__,
            EnsembleMethods.LOGISTIC_AVERAGE : self.__average__,
            EnsembleMethods.MAJORITY_VOTE : self.__average__
        }
        self.__ensemble_method__ = ensemble_types.get(ensemble_type, None)
        if self.__ensemble_method__ is None:
            raise ValueError("Invalid ensemble type")

    def call(self, x, *args, **kwargs):
        return self.__ensemble_method__(x)

    def get_all_predictions(self, x, verbose="auto"):
        return tf.stack([model.predict(x, verbose=verbose) for model in self.models])

    def get_all_votes(self, x, verbose="auto"):
        return tf.stack([model.predict(x, verbose=verbose).argmax(axis=-1) for model in self.models])

    def __average__(self, x, verbose="auto"):
        pred = self.get_all_predictions(x, verbose)
        print('PREDICTIONS shape', str(pred.shape))
        print('PREDICTIONS __average__', str(pred))
        max_pred = np.average(pred, axis=0, weights=self.model_weights)
        print('MAX PREDICTIONS shape', str(max_pred.shape))
        print('MAX PREDICTIONS __average__', str(max_pred))
        votes = self.get_all_votes(x, verbose)
        print('VOTES __average__', str(votes))
        print('VOTES shape', str(votes.shape))
        print("------------------------------------------")
        return tf.argmax(np.average(pred, axis=0, weights=self.model_weights), axis=-1)

    def __logistic_average__(self, x, verbose="auto"):
        pred = self.get_all_predictions(x, verbose)
        #print('PREDICTIONS __logistic_average__', str(pred))
        #print('VOTES __average__', str(self.get_all_votes(x, verbose)))
        return tf.argmax(np.average(tf.math.sigmoid(pred), axis=0, weights=self.model_weights), axis=-1)

    def __majority_vote__(self, x, verbose="auto"):
        return thread_first(self.get_all_votes(x, verbose),
                            tf.stack,
                            (weighted_mode, np.transpose(np.array([self.model_weights]*len(x)))),  # axis is 0 by default
                            first,
                            first)


In [32]:
def get_ensemble_predictions(x, models, evaluation_dataset, check_identical=False):
    """
    Create an ensemble of given models and dataset

    :param x: prediction input
    :param models: models to be ensembled
    :return: ensemble methods used and class predictions
    """
    models, weights = list(zip(*flatten_models(models)))
    print(models)
    print(weights)
    models = load_models(evaluation_dataset, models, model_path="../../../models")
    ensemble_methods = [method.value for method in EnsembleMethods]
    #ensembles = list(map(lambda ensemble_type: Ensemble(models=models, weights=weights, ensemble_type=ensemble_type),
                         #ensemble_methods))
    ensembles = [Ensemble(models=models, weights=weights, ensemble_type=EnsembleMethods.AVERAGE)]
    predictions = list(map(lambda ensemble: np.array(ensemble.__ensemble_method__(x, verbose=0)),
                           ensembles))
    # check for identical predictions
    if check_identical:
        identical_predictions_count = thread_last(combinations(predictions, 2),
                                                  (map, lambda comb: array_equal(comb[0], comb[1])),
                                                  (filter, identity),
                                                  count)
        print(f"{identical_predictions_count} identical predictions found")
    return ensemble_methods, predictions

In [33]:

def run_ensemble(evaluation_dataset=CurrentDatasets.swedish_leaf.value, model_names=['Encoder'], ensemble_name=None):
    """
    run an ensemble on a given dataset

    :param evaluation_dataset: dataset name
    :param model_names: model names to be ensembled, can be a nested list. Nested lists get weighted
    :param ensemble_name
    :return: list of accuracies and confusion matrices
    """
    if ensemble_name is None:
        ensemble_name = str(model_names)

    x_test, y_test = get_all_datasets_test_train_np_arrays('../../../datasets/', [evaluation_dataset])[evaluation_dataset][
        'test_data']
    
    print("X_TEST SIZE", str(x_test.shape))

    method_names, predicted_classes = get_ensemble_predictions(x_test, model_names, evaluation_dataset)

    dataset_names = [evaluation_dataset for _ in method_names]
    display_names = [f"{ensemble_name}-{method_name}" for method_name in method_names]
    accuracies = list(map(partial(accuracy_score, y_test), predicted_classes))
    confusion_matrices = list(map(lambda predicted: json.dumps(confusion_matrix(y_test, predicted).tolist()),
                                  predicted_classes))
    return list(zip(dataset_names, display_names, accuracies, confusion_matrices))

In [34]:
dataset_names = [dataset.value for dataset in CurrentDatasets]
v = dataset_names[:1]
ensembles = {"Encoder_MLP": ["Encoder","MLP"],}
csv_name = './test_Ensembles.csv'

result = pd.DataFrame(columns=['dataset_name', 'model_name', 'test_acc', 'confusion_matrix'])
i = 1
verbose = False
for dataset_name in dataset_names:
    if verbose:
        print(f"{i}/{len(dataset_names)}:\t{dataset_name}")
    for ensemble_name, model_names in ensembles.items():
        for row in run_ensemble(dataset_name, model_names, ensemble_name):
            result.loc[len(result)] = row
    i = i + 1
result.to_csv(csv_name)
print(result)

X_TEST SIZE (390, 300)
('Encoder', 'MLP')
(0.5, 0.5)


/opt/bwhpc/common/jupyter/tensorflow/2023-01-02/lib/python3.8/site-packages/keras/layers/core/lambda_layer.py:324: UserWarning: ModelBuilder is not loaded, but a Lambda layer uses it. It may cause errors.
  function = cls._parse_function_from_config(


PREDICTIONS shape (2, 390, 12)
PREDICTIONS __average__ tf.Tensor(
[[[1.21165685e-01 2.65879165e-02 4.04902101e-02 ... 6.35874495e-02
   4.79618981e-02 5.44390269e-02]
  [3.57979238e-02 1.23451084e-01 6.42736033e-02 ... 1.00678362e-01
   1.56364396e-01 3.64242829e-02]
  [8.39434415e-02 1.26379013e-01 1.35067642e-01 ... 2.62827147e-02
   1.97106060e-02 2.05780387e-01]
  ...
  [1.03258155e-01 6.08761311e-02 5.42324521e-02 ... 1.02561615e-01
   5.51136807e-02 7.07389712e-02]
  [4.20142449e-02 3.34474146e-02 4.78862561e-02 ... 1.54805362e-01
   1.52072489e-01 3.14081274e-02]
  [5.23866117e-02 4.54358272e-02 4.94200252e-02 ... 1.50252745e-01
   1.96513802e-01 4.21646610e-02]]

 [[1.72483742e-05 3.32792274e-06 4.88372365e-09 ... 8.40083667e-05
   1.17030659e-05 9.72693215e-07]
  [1.04376868e-13 7.17010609e-12 9.82255235e-15 ... 5.84301894e-14
   5.20504965e-15 8.90268304e-09]
  [3.07578518e-10 3.69926092e-05 1.41379161e-07 ... 3.87823746e-07
   1.67385905e-09 9.99732614e-01]
  ...
  [9.776392

/opt/bwhpc/common/jupyter/tensorflow/2023-01-02/lib/python3.8/site-packages/keras/layers/core/lambda_layer.py:324: UserWarning: ModelBuilder is not loaded, but a Lambda layer uses it. It may cause errors.
  function = cls._parse_function_from_config(


PREDICTIONS shape (2, 139, 6)
PREDICTIONS __average__ tf.Tensor(
[[[1.0597374e-01 1.2912266e-01 4.5530573e-01 1.4109349e-01 7.2142690e-02
   9.6361659e-02]
  [9.1455661e-02 1.2231769e-01 5.0434291e-01 1.3692406e-01 6.6152014e-02
   7.8807600e-02]
  [1.6761407e-02 1.0766396e-02 1.1637848e-02 7.0932754e-03 6.1440892e-03
   9.4759697e-01]
  ...
  [1.0361757e-01 1.0752160e-01 2.7132490e-01 9.0757288e-02 5.5499677e-02
   3.7127894e-01]
  [7.5808503e-02 6.9381550e-02 1.4535354e-01 5.3707048e-02 3.8555022e-02
   6.1719429e-01]
  [1.0531725e-01 1.2260758e-01 3.9150709e-01 1.1599881e-01 6.4424917e-02
   2.0014434e-01]]

 [[1.4004716e-01 6.6771263e-01 1.5978611e-01 1.8908657e-02 4.6586054e-03
   8.8868402e-03]
  [9.8587172e-03 2.4131021e-01 5.8653986e-01 1.3042973e-01 2.9101688e-02
   2.7597230e-03]
  [6.0004590e-04 1.5520753e-05 8.6893357e-07 1.1187993e-06 8.6802924e-05
   9.9929571e-01]
  ...
  [9.8101908e-01 1.6975693e-02 2.6084596e-04 5.3920189e-06 7.6859578e-06
   1.7312283e-03]
  [9.622336

...

PREDICTIONS shape (2, 20, 2)
PREDICTIONS __average__ tf.Tensor(
[[[2.4293990e-04 9.9975705e-01]
  [9.9999565e-01 4.2965630e-06]
  [2.5068868e-05 9.9997491e-01]
  [9.9510335e-06 9.9999005e-01]
  [6.5814716e-01 3.4185281e-01]
  [1.4959524e-05 9.9998504e-01]
  [2.5413950e-05 9.9997455e-01]
  [2.9662417e-05 9.9997026e-01]
  [4.4598448e-05 9.9995536e-01]
  [9.9999994e-01 5.2560969e-09]
  [3.3060958e-05 9.9996692e-01]
  [9.9999732e-01 2.6150137e-06]
  [1.7694554e-05 9.9998230e-01]
  [7.9490419e-05 9.9992043e-01]
  [5.8116311e-05 9.9994177e-01]
  [3.2099738e-04 9.9967897e-01]
  [3.6189130e-03 9.9638110e-01]
  [2.1896546e-03 9.9781042e-01]
  [9.9908519e-01 9.1485254e-04]
  [1.4931233e-05 9.9998510e-01]]

 [[1.5200680e-04 9.9984813e-01]
  [9.9807644e-01 1.9235316e-03]
  [1.2515750e-03 9.9874848e-01]
  [1.7749738e-04 9.9982262e-01]
  [8.3035600e-01 1.6964392e-01]
  [6.6608663e-06 9.9999326e-01]
  [1.9358904e-05 9.9998063e-01]
  [1.1259350e-05 9.9998873e-01]
  [3.4035133e-05 9.9996585e-01]
  [9.9

/opt/bwhpc/common/jupyter/tensorflow/2023-01-02/lib/python3.8/site-packages/keras/layers/core/lambda_layer.py:324: UserWarning: ModelBuilder is not loaded, but a Lambda layer uses it. It may cause errors.
  function = cls._parse_function_from_config(


PREDICTIONS shape (2, 625, 15)
PREDICTIONS __average__ tf.Tensor(
[[[5.8172051e-02 3.2840256e-02 7.2137542e-02 ... 5.5326585e-02
   1.0249608e-01 7.7437699e-02]
  [7.1304120e-02 1.6572498e-02 7.8446023e-02 ... 7.0387274e-02
   9.6427761e-02 7.2575204e-02]
  [3.1017754e-02 9.1653928e-02 4.7398679e-02 ... 7.0862629e-02
   6.5220661e-02 5.9223939e-02]
  ...
  [2.1404438e-02 2.7554592e-01 2.9311122e-02 ... 4.3642350e-02
   2.2397602e-02 4.4149023e-02]
  [6.3993961e-02 2.4162896e-02 6.0484581e-02 ... 1.0059272e-01
   7.4935310e-02 8.7302729e-02]
  [7.4038513e-02 1.2000686e-02 9.3570828e-02 ... 5.4931443e-02
   1.1107586e-01 5.9965726e-02]]

 [[1.2146010e-02 3.6032118e-02 6.5111713e-03 ... 5.9202234e-03
   2.0966869e-02 1.6804956e-03]
  [1.2197417e-05 4.4152817e-08 6.4582599e-04 ... 1.9030240e-05
   5.4257005e-05 8.5717300e-03]
  [3.7727822e-03 2.7027896e-01 1.9944265e-01 ... 1.9337485e-02
   5.7289103e-04 2.1936018e-03]
  ...
  [4.2756629e-04 9.9944484e-01 1.6386148e-06 ... 6.1393905e-08
  

/opt/bwhpc/common/jupyter/tensorflow/2023-01-02/lib/python3.8/site-packages/keras/layers/core/lambda_layer.py:324: UserWarning: ModelBuilder is not loaded, but a Lambda layer uses it. It may cause errors.
  function = cls._parse_function_from_config(


PREDICTIONS shape (2, 300, 6)
PREDICTIONS __average__ tf.Tensor(
[[[2.69890338e-01 1.98818937e-01 1.31090045e-01 2.23554045e-01
   1.41765729e-01 3.48809734e-02]
  [2.55094707e-01 1.90186828e-01 2.00755224e-01 1.01921432e-01
   2.13308811e-01 3.87330242e-02]
  [2.67897725e-01 3.00627887e-01 1.45436570e-01 1.24609686e-01
   1.32925078e-01 2.85029560e-02]
  ...
  [1.56548977e-01 1.99948534e-01 3.42781432e-02 5.52955329e-01
   3.83451842e-02 1.79238059e-02]
  [9.28539261e-02 8.85111541e-02 1.52831860e-02 7.83557236e-01
   9.02404822e-03 1.07704606e-02]
  [1.08842388e-01 1.13147296e-01 2.13841554e-02 7.30357409e-01
   1.61286499e-02 1.01400176e-02]]

 [[9.98588800e-01 9.46952205e-04 1.54635840e-04 1.64059529e-04
   1.31334382e-04 1.41766213e-05]
  [9.97526884e-01 4.21631470e-04 3.29897099e-04 2.47659511e-04
   1.44123996e-03 3.25776891e-05]
  [7.06356391e-02 9.26642478e-01 7.02677295e-04 1.61223148e-03
   3.29691451e-04 7.72328640e-05]
  ...
  [4.93170647e-03 1.39156659e-03 5.32858285e-05 

In [63]:
pred_MLP = [0.9, 0.05, 0.05]
pred_Encoder = [0.1, 0.8, 0.1]
model_weights = [1, 9]
preds = np.array([pred_MLP, pred_Encoder])

### original average

In [64]:
np.average(preds, axis=0, weights = model_weights)

array([0.18 , 0.725, 0.095])

### consider the confidence

In [61]:
def conf_avg(pred_column):
    return np.average(pred_column, axis=0, weights = model_weights*pred_column*10)

In [62]:
np.apply_along_axis(conf_avg, 0, preds)

array([0.5       , 0.79482759, 0.09736842])

### without model weights

In [67]:
def conf_avg(pred_column):
    return np.average(pred_column, axis=0, weights = pred_column*10)

In [68]:
np.apply_along_axis(conf_avg, 0, preds)

array([0.82      , 0.75588235, 0.08333333])